# LDED Audiovisual Fusion 

Author: Chen Lequn.
Created on 10 Oct 2023.

- Material: Maraging Steel 300
- Process: Robotic Llser-directed energy deposition
- Recorded data: position, veolocity, coaxial ccd features, acoustic feature
- Quality labels generated: keyhole pores, cracks, defect-free

### Notebook 2b: Spatio-temporal feature registration
- __Register features, image, audio slices, and their features with the corresponding location/velocity data on the same  time stamps__ 


In [1]:
import cv2
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict

## plot
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
%matplotlib inline
import seaborn as sns

In [2]:
PROJECT_ROOT_DIR = "../"
IMAGE_PATH = os.path.join(PROJECT_ROOT_DIR, "result_images", 'feature_extraction')
os.makedirs(IMAGE_PATH, exist_ok=True)

Multimodal_dataset_PATH = "/home/chenlequn/Dataset/LDED_acoustic_visual_monitoring_dataset"
Dataset_path = os.path.join(Multimodal_dataset_PATH, f'25Hz')
                            

## function for automatically save the diagram/graph into the folder 
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGE_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 2.50

In [3]:
def get_sample_directories(base_path, sample_numbers):
    sample_directories = []
    for sample_number in sample_numbers:
        sample_directories.append(os.path.join(base_path, f'{sample_number}'))
    return sample_directories


samples = [21, 22, 23, 24, 26, 32]
sample_directories = get_sample_directories(Dataset_path, samples)

# Get lists of image and audio directories for each sample
image_directories = [os.path.join(sample_dir, 'images') for sample_dir in sample_directories]
audio_directories = [os.path.join(sample_dir, 'raw_audio') for sample_dir in sample_directories]

In [4]:
# Combine all annotation files into one DataFrame
all_annotation_dfs = []
for sample_dir, sample_number in zip(sample_directories, samples):
    annotation_file = os.path.join(sample_dir, f'annotations_{sample_number}.csv')  # Update the file name
    annotation_df = pd.read_csv(annotation_file)
    all_annotation_dfs.append(annotation_df)
combined_annotation_df = pd.concat(all_annotation_dfs)
combined_annotation_df

,sample index,audio_file_name,image_file_name,class_name,class_name_v2,Layer number,Sample number,class_name_v3
0,1,sample_21_1.wav,sample_21_1.jpg,Laser-off,Laser-off,1.0,21,Laser-off
1,2,sample_21_2.wav,sample_21_2.jpg,Defect-free,Defect-free,1.0,21,Defect-free
2,3,sample_21_3.wav,sample_21_3.jpg,Defect-free,Defect-free,1.0,21,Defect-free
3,4,sample_21_4.wav,sample_21_4.jpg,Defect-free,Defect-free,1.0,21,Defect-free
4,5,sample_21_5.wav,sample_21_5.jpg,Defect-free,Defect-free,1.0,21,Defect-free
...,...,...,...,...,...,...,...,...
13523,13524,sample_32_13524.wav,sample_32_13524.jpg,NaN,NaN,NaN,32,NaN
13524,13525,sample_32_13525.wav,sample_32_13525.jpg,NaN,NaN,NaN,32,NaN
13525,13526,sample_32_13526.wav,sample_32_13526.jpg,NaN,NaN,NaN,32,NaN
13526,13527,sample_32_13527.wav,sample_32_13527.jpg,NaN,NaN,NaN,32,NaN


In [5]:
# Read the data back from the HDF5 file into a new DataFrame
df_multimodal = pd.read_hdf(os.path.join(Dataset_path, 'data_audiovisual_with_annotations(raw_audio).h5'), key='df')
# df_multimodal = df_multimodal.dropna(subset=['class_name'])
df_multimodal

,sample index,audio_file_name,image_file_name,class_name,class_name_v2,Layer number,Sample number,class_name_v3,rms_energy,amplitude_envelope_mean,...,nu11,nu02,nu30,nu21,nu12,nu03,center_x,center_y,contour_area,contour_length
0,1,sample_21_1.wav,sample_21_1.jpg,Laser-off,Laser-off,1.0,21,Laser-off,0.009018,0.003034,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,2,sample_21_2.wav,sample_21_2.jpg,Defect-free,Defect-free,1.0,21,Defect-free,0.019286,0.012109,...,0.000000,0.062337,0.000000,0.000000,0.000000,0.000000,319.500000,239.000000,305442.0,2234.000000
2,3,sample_21_3.wav,sample_21_3.jpg,Defect-free,Defect-free,1.0,21,Defect-free,0.019593,0.015114,...,0.000000,0.062337,0.000000,0.000000,0.000000,0.000000,319.500000,239.000000,305442.0,2234.000000
3,4,sample_21_4.wav,sample_21_4.jpg,Defect-free,Defect-free,1.0,21,Defect-free,0.030937,0.021501,...,-0.008049,0.062216,0.001940,-0.002304,-0.001599,0.000891,307.440996,230.264496,291865.5,2279.781744
4,5,sample_21_5.wav,sample_21_5.jpg,Defect-free,Defect-free,1.0,21,Defect-free,0.038329,0.029851,...,-0.009879,0.065910,0.001738,-0.003345,-0.000898,0.001388,296.636877,229.109962,281970.0,2407.847760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61989,13524,sample_32_13524.wav,sample_32_13524.jpg,NaN,NaN,NaN,32,NaN,0.048554,0.036195,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
61990,13525,sample_32_13525.wav,sample_32_13525.jpg,NaN,NaN,NaN,32,NaN,0.040282,0.032946,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
61991,13526,sample_32_13526.wav,sample_32_13526.jpg,NaN,NaN,NaN,32,NaN,0.052285,0.042469,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
61992,13527,sample_32_13527.wav,sample_32_13527.jpg,NaN,NaN,NaN,32,NaN,0.023477,0.016409,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


# Data cleaning

## Load a single position dataset for inspection and cleaning

In [6]:
# Load the CSV file into a Pandas DataFrame
file_path = os.path.join(Dataset_path, '21', 'position_21_raw.csv') 
df_position_21_raw = pd.read_csv(file_path)
df_position_21_raw

,Time_1,field.X,field.Y,field.Z,field.A,field.B,field.C,%time,Time_2,field.Vx,field.Vy,field.Vz,field.Speed
0,0.00000,2981.842041,-1712.173584,918.833313,-90.106903,-0.5406,178.034195,1.655970e+18,0.00000,0.000002,-0.000001,5.550000e-07,0.000002
1,0.00153,2981.842529,-1712.173096,918.833313,-90.106903,-0.5406,178.034195,1.655970e+18,0.00153,0.126762,0.126762,0.000000e+00,0.179269
2,0.00547,2981.841797,-1712.173950,918.833618,-90.106903,-0.5406,178.034195,1.655970e+18,0.00548,-0.179277,-0.230498,7.683282e-02,0.301949
3,0.00969,2981.842285,-1712.173462,918.833496,-90.106903,-0.5406,178.034302,1.655970e+18,0.00971,0.126038,0.126038,-2.520767e-02,0.180019
4,0.01364,2981.842773,-1712.173340,918.833618,-90.106903,-0.5406,178.034195,1.655970e+18,0.01365,0.120091,0.048036,2.401823e-02,0.131553
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51820,207.27046,2981.833008,-1712.172852,938.404175,-90.106796,-0.5406,178.033096,1.655970e+18,207.27046,0.048675,0.073012,2.433738e-02,0.091062
51821,207.27442,2981.833008,-1712.172852,938.404175,-90.106697,-0.5406,178.033096,1.655970e+18,207.27442,0.000000,0.000000,0.000000e+00,0.000000
51822,207.27849,2981.832520,-1712.173340,938.404175,-90.106796,-0.5406,178.033096,1.655970e+18,207.27851,-0.125593,-0.100474,0.000000e+00,0.160838
51823,207.28420,2981.832764,-1712.172974,938.404175,-90.106796,-0.5406,178.033096,1.655970e+18,207.28420,0.073425,0.073425,0.000000e+00,0.103839


1. Remove the columns A, B, and C.
2. Synchronize the time axes __Time_1__ and __Time_2__.
3. Rename the columns to remove the "field" prefix and unify the dataset.
4. Offset the X,Y,Z positions by the value of the first row.
5. Move the synchronized time column to the first column in the DataFrame.

### time-sync algorithm: 

- finds the closest time stamp from the second column for each entry in the first column.
- Specifically, it iterates through each time stamp in 'Time_1' and finds the closest time stamp in 'Time_2'.
- The closest time stamp is considered the best approximate sync point.

### Function to process raw dataframe

In [7]:
def process_raw_dataframe(df_raw):
    # Step 2: Remove the columns A, B, and C
    df_cleaned = df_raw.drop(columns=['field.A', 'field.B', 'field.C', '%time'])

    # Use Time_1 as Time_Stamp_Spatial
    df_cleaned.rename(columns={'Time_1': 'Time_Stamp_Spatial'}, inplace=True)
    df_cleaned.drop(columns=['Time_2'], inplace=True)

    # Step 4: Offset the X, Y, Z positions by the value of the first row
    first_row = df_cleaned.iloc[0]
    df_cleaned['field.X'] -= first_row['field.X']
    df_cleaned['field.Y'] -= first_row['field.Y']
    df_cleaned['field.Z'] -= first_row['field.Z']

    # Step 5: Rename the columns
    column_rename_map = {
        'field.X': 'X',
        'field.Y': 'Y',
        'field.Z': 'Z',
        'field.Vx': 'Vx',
        'field.Vy': 'Vy',
        'field.Vz': 'Vz',
        'field.Speed': 'Speed'
    }
    df_cleaned.rename(columns=column_rename_map, inplace=True)

    # Step 6: Move the synchronized time column to be the first column
    column_order = ['Time_Stamp_Spatial'] + [col for col in df_cleaned.columns if col != 'Time_Stamp_Spatial']
    df_cleaned = df_cleaned[column_order]

    # Step 7: Round the values to 3 decimal places
    columns_to_round = ['X', 'Y', 'Z', 'Vx', 'Vy', 'Vz', 'Speed']
    df_cleaned[columns_to_round] = df_cleaned[columns_to_round].round(3)

    return df_cleaned

In [8]:
# Test the corrected function with the provided CSV file
df_processed_21 = process_raw_dataframe(df_position_21_raw)
df_processed_21

,Time_Stamp_Spatial,X,Y,Z,Vx,Vy,Vz,Speed
0,0.00000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000
1,0.00153,0.000,0.000,0.000,0.127,0.127,0.000,0.179
2,0.00547,-0.000,-0.000,0.000,-0.179,-0.230,0.077,0.302
3,0.00969,0.000,0.000,0.000,0.126,0.126,-0.025,0.180
4,0.01364,0.001,0.000,0.000,0.120,0.048,0.024,0.132
...,...,...,...,...,...,...,...,...
51820,207.27046,-0.009,0.001,19.571,0.049,0.073,0.024,0.091
51821,207.27442,-0.009,0.001,19.571,0.000,0.000,0.000,0.000
51822,207.27849,-0.010,0.000,19.571,-0.126,-0.100,0.000,0.161
51823,207.28420,-0.009,0.001,19.571,0.073,0.073,0.000,0.104


In [9]:
# df_processed_21[:1000].plot('Time_Stamp_Spatial', 'Vx')

### Load annotation file for processing

In [9]:
# Step 1: Read the example annotation file into a Pandas DataFrame
annotation_21_df = pd.read_csv(os.path.join(Dataset_path, '21', 'annotations_21.csv') )

# # Add a time stamps column based on the sample index, each sample corresponds to a 40 ms segment
# # Convert sample index to time stamps in ms
# annotation_21_df['Time_Stamps'] = (annotation_21_df['sample index'] - 1) * 0.04

# cols = annotation_21_df.columns.tolist()
# cols.insert(1, cols.pop(cols.index('Time_Stamps')))
# annotation_21_df = annotation_21_df[cols]

# # Show the first few rows of the annotation DataFrame
# annotation_21_df

In [10]:
# Function to get the indices of synchronized time stamps in the high-frequency DataFrame
def get_synchronized_indices(time_high_freq, time_low_freq):
    if isinstance(time_high_freq, pd.Series):
        time_high_freq = time_high_freq.to_numpy()
    if isinstance(time_low_freq, pd.Series):
        time_low_freq = time_low_freq.to_numpy()
        
    synchronized_indices = np.zeros_like(time_low_freq, dtype=int)
    j = 0
    
    for i in range(len(time_low_freq)):
        min_diff = float('inf')
        while j < len(time_high_freq):
            diff = abs(time_low_freq[i] - time_high_freq[j])
            if diff < min_diff:
                min_diff = diff
                synchronized_indices[i] = j
            else:
                break
            j += 1
            
    return synchronized_indices

In [27]:
# Get the synchronized indices
# synchronized_indices = get_synchronized_indices(df_processed_21['Time_Stamp_Spatial'], annotation_21_df['Time_Stamps'])

# # Extract the corresponding rows from the high-frequency DataFrame
# df_synchronized_position = df_processed_21.iloc[synchronized_indices].reset_index(drop=True)

# # Combine the annotation DataFrame and the synchronized position DataFrame
# df_final = pd.concat([annotation_21_df, df_synchronized_position.drop(columns=['Time_Stamp_Spatial'])], axis=1)

# # Adjust the column order
# column_order = ['sample index', 'Time_Stamps'] + [col for col in df_final.columns if col not in ['sample index', 'Time_Stamps']]
# df_final = df_final[column_order]

# df_final

In [19]:
# df_final.plot('Time_Stamps', 'Speed')

## Data processing pipeline for individual sample

In [11]:
# Function to create the final consolidated DataFrame
def create_consolidated_dataframe(annotation_df, position_df):
    # Step 1: Process the raw position DataFrame to get the modified DataFrame
    df_processed_position = process_raw_dataframe(position_df)
    annotation_df['Time_Stamps'] = (annotation_df['sample index'] - 1) * 0.04
    
    # Step 2: Get the synchronized indices
    synchronized_indices = get_synchronized_indices(df_processed_position['Time_Stamp_Spatial'], annotation_df['Time_Stamps'])
    
    # Step 3: Extract the corresponding rows from the processed position DataFrame
    df_synchronized_position = df_processed_position.iloc[synchronized_indices].reset_index(drop=True)
    
    # Step 4: Combine the annotation DataFrame and the synchronized position DataFrame
    df_final = pd.concat([annotation_df, df_synchronized_position.drop(columns=['Time_Stamp_Spatial'])], axis=1)
    
    # Step 5: Adjust the column order
    column_order = ['sample index', 'Time_Stamps'] + [col for col in df_final.columns if col not in ['sample index', 'Time_Stamps']]
    df_final = df_final[column_order]
    
    return df_final

In [12]:
# Test the pipeline function
df_final_test = create_consolidated_dataframe(annotation_21_df, df_position_21_raw)
df_final_test

,sample index,Time_Stamps,audio_file_name,image_file_name,class_name,class_name_v2,Layer number,Sample number,class_name_v3,X,Y,Z,Vx,Vy,Vz,Speed
0,1,0.00,sample_21_1.wav,sample_21_1.jpg,Laser-off,Laser-off,1.0,21,Laser-off,0.000,0.000,0.000,0.000,-0.000,0.000,0.000
1,2,0.04,sample_21_2.wav,sample_21_2.jpg,Defect-free,Defect-free,1.0,21,Defect-free,0.000,-0.000,0.001,-0.074,-0.074,0.000,0.105
2,3,0.08,sample_21_3.wav,sample_21_3.jpg,Defect-free,Defect-free,1.0,21,Defect-free,-0.010,0.015,-0.007,-0.051,0.026,-0.051,0.077
3,4,0.12,sample_21_4.wav,sample_21_4.jpg,Defect-free,Defect-free,1.0,21,Defect-free,-0.007,0.016,0.010,0.173,-0.222,2.520,2.536
4,5,0.16,sample_21_5.wav,sample_21_5.jpg,Defect-free,Defect-free,1.0,21,Defect-free,-0.111,0.123,-0.057,-1.864,1.662,-2.468,3.511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5138,5139,205.52,sample_21_5139.wav,sample_21_5139.jpg,Laser-off,Laser-off,NaN,21,Laser-off,-0.005,-0.002,19.578,-0.049,-0.073,0.024,0.092
5139,5140,205.56,sample_21_5140.wav,sample_21_5140.jpg,Laser-off,Laser-off,NaN,21,Laser-off,-0.006,-0.002,19.578,0.000,-0.059,-0.020,0.062
5140,5141,205.60,sample_21_5141.wav,sample_21_5141.jpg,Laser-off,Laser-off,NaN,21,Laser-off,-0.005,-0.001,19.578,0.127,0.000,0.152,0.198
5141,5142,205.64,sample_21_5142.wav,sample_21_5141.jpg,Laser-off,Laser-off,NaN,21,Laser-off,-0.006,-0.001,19.577,-0.074,0.000,-0.025,0.078


### Data processing pipeline for each samples

1. Separate this big DataFrame (multimodal features) into individual sample DataFrames.
2. Apply the synchronization pipeline for each sample to combine the position information with the existing sensor data features.
3. Reassemble the synchronized DataFrames back into one large DataFrame.

In [13]:
# Separate the big DataFrame into individual sample DataFrames based on the 'Sample number' column
sample_numbers = [21, 22, 23, 24, 26, 32]
sample_dfs = {sample: df_multimodal[df_multimodal['Sample number'] == sample].reset_index(drop=True) for sample in sample_numbers}

# Display the shape of each individual sample DataFrame to verify the separation
{sample: sample_df.shape for sample, sample_df in sample_dfs.items()}

{21: (5143, 127),
 22: (4624, 127),
 23: (11396, 127),
 24: (16317, 127),
 26: (10986, 127),
 32: (13528, 127)}

In [14]:
for sample, df in sample_dfs.items():
    df['Time_Stamps'] = (df['sample index'] - 1) * 0.04
    
    # Move Time_Stamps column to the front, right after 'sample index'
    cols = df.columns.tolist()
    cols.insert(1, cols.pop(cols.index('Time_Stamps')))
    sample_dfs[sample] = df[cols]

In [15]:
# Display the first few rows of one of the modified DataFrames to verify
sample_dfs[21]

,sample index,Time_Stamps,audio_file_name,image_file_name,class_name,class_name_v2,Layer number,Sample number,class_name_v3,rms_energy,...,nu11,nu02,nu30,nu21,nu12,nu03,center_x,center_y,contour_area,contour_length
0,1,0.00,sample_21_1.wav,sample_21_1.jpg,Laser-off,Laser-off,1.0,21,Laser-off,0.009018,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,2,0.04,sample_21_2.wav,sample_21_2.jpg,Defect-free,Defect-free,1.0,21,Defect-free,0.019286,...,0.000000,0.062337,0.000000,0.000000,0.000000,0.000000,319.500000,239.000000,305442.0,2234.000000
2,3,0.08,sample_21_3.wav,sample_21_3.jpg,Defect-free,Defect-free,1.0,21,Defect-free,0.019593,...,0.000000,0.062337,0.000000,0.000000,0.000000,0.000000,319.500000,239.000000,305442.0,2234.000000
3,4,0.12,sample_21_4.wav,sample_21_4.jpg,Defect-free,Defect-free,1.0,21,Defect-free,0.030937,...,-0.008049,0.062216,0.001940,-0.002304,-0.001599,0.000891,307.440996,230.264496,291865.5,2279.781744
4,5,0.16,sample_21_5.wav,sample_21_5.jpg,Defect-free,Defect-free,1.0,21,Defect-free,0.038329,...,-0.009879,0.065910,0.001738,-0.003345,-0.000898,0.001388,296.636877,229.109962,281970.0,2407.847760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5138,5139,205.52,sample_21_5139.wav,sample_21_5139.jpg,Laser-off,Laser-off,NaN,21,Laser-off,0.022243,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5139,5140,205.56,sample_21_5140.wav,sample_21_5140.jpg,Laser-off,Laser-off,NaN,21,Laser-off,0.019228,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5140,5141,205.60,sample_21_5141.wav,sample_21_5141.jpg,Laser-off,Laser-off,NaN,21,Laser-off,0.025485,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5141,5142,205.64,sample_21_5142.wav,sample_21_5141.jpg,Laser-off,Laser-off,NaN,21,Laser-off,0.018279,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [16]:
final_dfs = {}

# Iterate over each sample number to perform the synchronization
for sample in sample_numbers:
    # Load the corresponding raw position DataFrame
    position_df = pd.read_csv(os.path.join(Dataset_path, f'{sample}', f'position_{sample}_raw.csv') )
    
    # Get the individual sample DataFrame from the big DataFrame
    individual_sample_df = sample_dfs[sample]
    
    # Apply the synchronization pipeline to get the final DataFrame for this sample
    final_df = create_consolidated_dataframe(individual_sample_df, position_df)
    
    # Store the final DataFrame in the dictionary
    final_dfs[sample] = final_df

# Display the shape of each final DataFrame to verify the synchronization
{sample: final_df.shape for sample, final_df in final_dfs.items()}

{21: (5143, 135),
 22: (4624, 135),
 23: (11396, 135),
 24: (16317, 135),
 26: (10986, 135),
 32: (13528, 135)}

In [17]:
final_dfs[22]

,sample index,Time_Stamps,audio_file_name,image_file_name,class_name,class_name_v2,Layer number,Sample number,class_name_v3,rms_energy,...,center_y,contour_area,contour_length,X,Y,Z,Vx,Vy,Vz,Speed
0,1,0.00,sample_22_1.wav,sample_22_1.jpg,Defect-free,Defect-free,1.0,22,Defect-free,0.017784,...,239.000000,305442.0,2234.000000,0.000,0.000,0.000,-0.075,-0.050,0.000,0.090
1,2,0.04,sample_22_2.wav,sample_22_2.jpg,Defect-free,Defect-free,1.0,22,Defect-free,0.023733,...,239.000000,305442.0,2234.000000,0.000,-0.001,0.001,0.075,-0.125,0.150,0.209
2,3,0.08,sample_22_3.wav,sample_22_3.jpg,Defect-free,Defect-free,1.0,22,Defect-free,0.027532,...,238.900284,305313.5,2331.899495,-0.000,-0.001,0.001,0.000,0.051,-0.051,0.072
3,4,0.12,sample_22_4.wav,sample_22_4.jpg,Defect-free,Defect-free,1.0,22,Defect-free,0.024417,...,231.762180,239222.5,2417.428494,-0.000,-0.001,0.002,0.048,0.000,0.000,0.048
4,5,0.16,sample_22_5.wav,sample_22_5.jpg,Defect-free,Defect-free,1.0,22,Defect-free,0.034348,...,235.962684,219186.5,2391.737149,-0.000,-0.002,0.003,0.049,0.099,-0.099,0.148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4619,4620,184.76,sample_22_4620.wav,sample_22_4620.jpg,Keyhole pores,Defective,50.0,22,Keyhole pores,0.033907,...,235.457420,269669.5,2333.948265,38.439,-0.008,19.630,NaN,NaN,NaN,NaN
4620,4621,184.80,sample_22_4621.wav,sample_22_4621.jpg,Keyhole pores,Defective,50.0,22,Keyhole pores,0.034723,...,234.973194,194352.0,1931.577769,37.334,-0.001,19.627,NaN,NaN,NaN,NaN
4621,4622,184.84,sample_22_4622.wav,sample_22_4622.jpg,Keyhole pores,Defective,50.0,22,Keyhole pores,0.034423,...,233.024406,222826.0,2207.612259,36.237,-0.002,19.628,NaN,NaN,NaN,NaN
4622,4623,184.88,sample_22_4623.wav,sample_22_4623.jpg,Keyhole pores,Defective,50.0,22,Keyhole pores,0.047280,...,230.293443,200884.0,2086.454966,35.141,-0.016,19.636,NaN,NaN,NaN,NaN


the synchronization pipeline has been successfully applied to each individual sample DataFrame. 
- Each resulting DataFrame has 134 columns: 126 columns from the original multimodal DataFrame plus 8 new columns for the position information (X, Y, Z, Vx, Vy, Vz, Speed, Time stamp.).

## Save the final consolidated dataset

In [19]:
# for col in ['audio_file_name', 'image_file_name', 'class_name', 'class_name_v2', 'class_name_v3']:
#     df_audiovisual[col] = df_audiovisual[col].astype('category')

In [20]:
final_dfs

{21:       sample index  Time_Stamps     audio_file_name     image_file_name  \
 0                1         0.00     sample_21_1.wav     sample_21_1.jpg   
 1                2         0.04     sample_21_2.wav     sample_21_2.jpg   
 2                3         0.08     sample_21_3.wav     sample_21_3.jpg   
 3                4         0.12     sample_21_4.wav     sample_21_4.jpg   
 4                5         0.16     sample_21_5.wav     sample_21_5.jpg   
 ...            ...          ...                 ...                 ...   
 5138          5139       205.52  sample_21_5139.wav  sample_21_5139.jpg   
 5139          5140       205.56  sample_21_5140.wav  sample_21_5140.jpg   
 5140          5141       205.60  sample_21_5141.wav  sample_21_5141.jpg   
 5141          5142       205.64  sample_21_5142.wav  sample_21_5141.jpg   
 5142          5143       205.68  sample_21_5143.wav  sample_21_5141.jpg   
 
        class_name class_name_v2  Layer number  Sample number class_name_v3  \
 0 

In [21]:
# Save the corrected individual DataFrames to separate h5 files using 'table' format
for sample_num, df in final_dfs.items():
    df.to_hdf(os.path.join(Dataset_path, f'{sample_num}', f'spatiotemporal_fused_{sample_num}.h5'), key='df', mode='w', format='table')

In [23]:
# Combine them into one big DataFrame and save it to an h5 file using 'table' format
final_consolidated_df = pd.concat(final_dfs.values(), ignore_index=True)
final_consolidated_df.to_hdf(os.path.join(Dataset_path, 'spatiotemporal_fused_multimodal.h5'), key='df', mode='w', format='table')

### Load the data and check

In [27]:
# Read the data back from the HDF5 file into a new DataFrame
df_multimodal_consolidated = pd.read_hdf(os.path.join(Dataset_path, 'spatiotemporal_fused_multimodal.h5'), key='df')
df_multimodal_consolidated = df_multimodal_consolidated.dropna(subset=['class_name_v3'])
df_multimodal_consolidated

,sample index,Time_Stamps,audio_file_name,image_file_name,class_name,class_name_v2,Layer number,Sample number,class_name_v3,rms_energy,...,center_y,contour_area,contour_length,X,Y,Z,Vx,Vy,Vz,Speed
0,1,0.00,sample_21_1.wav,sample_21_1.jpg,Laser-off,Laser-off,1.0,21,Laser-off,0.009018,...,0.000000,0.0,0.000000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000
1,2,0.04,sample_21_2.wav,sample_21_2.jpg,Defect-free,Defect-free,1.0,21,Defect-free,0.019286,...,239.000000,305442.0,2234.000000,0.000,-0.000,0.001,-0.074,-0.074,0.000,0.105
2,3,0.08,sample_21_3.wav,sample_21_3.jpg,Defect-free,Defect-free,1.0,21,Defect-free,0.019593,...,239.000000,305442.0,2234.000000,-0.010,0.015,-0.007,-0.051,0.026,-0.051,0.077
3,4,0.12,sample_21_4.wav,sample_21_4.jpg,Defect-free,Defect-free,1.0,21,Defect-free,0.030937,...,230.264496,291865.5,2279.781744,-0.007,0.016,0.010,0.173,-0.222,2.520,2.536
4,5,0.16,sample_21_5.wav,sample_21_5.jpg,Defect-free,Defect-free,1.0,21,Defect-free,0.038329,...,229.109962,281970.0,2407.847760,-0.111,0.123,-0.057,-1.864,1.662,-2.468,3.511
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48461,10982,439.24,sample_26_10982.wav,sample_26_10982.jpg,Laser-off,Laser-off,NaN,26,Laser-off,0.011000,...,0.000000,0.0,0.000000,0.000,0.000,0.000,-0.076,0.025,0.000,0.080
48462,10983,439.28,sample_26_10983.wav,sample_26_10983.jpg,Laser-off,Laser-off,NaN,26,Laser-off,0.008347,...,0.000000,0.0,0.000000,0.000,0.000,0.000,-0.076,0.025,0.000,0.080
48463,10984,439.32,sample_26_10984.wav,sample_26_10984.jpg,Laser-off,Laser-off,NaN,26,Laser-off,0.011081,...,0.000000,0.0,0.000000,0.000,0.000,0.000,-0.076,0.025,0.000,0.080
48464,10985,439.36,sample_26_10985.wav,sample_26_10985.jpg,Laser-off,Laser-off,NaN,26,Laser-off,0.011044,...,0.000000,0.0,0.000000,0.000,0.000,0.000,-0.076,0.025,0.000,0.080
